# Brief description of the problem and data 

Objective is to build a model that predicts which Tweets are about real disasters and which one’s aren’t.
In data 10,000 tweets were hand classified.

Results are evaluted based on F1 score of the model.

## ______________________________________

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns